In [1]:
import time
import re, ast
import numpy as np
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4040 --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.0,com.datastax.spark:spark-cassandra-connector_2.11:2.0.0-M3 pyspark-shell'



In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, Row
conf = SparkConf() \
    .setAppName("Streaming test") \
    .setMaster("local[2]") \
    .set("spark.cassandra.connection.host", "127.0.0.1") # "local[2]" to run locally with 2 cores
sc = SparkContext(conf=conf) 
sqlContext=SQLContext(sc)
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

In [3]:
ssc = StreamingContext(sc, 5) 
# 0.1 = batchDuration
# Main entry point for Spark Streaming functionality. A StreamingContext represents the
# connection to a Spark cluster, and can be used to create DStream various input sources.
# A Discretized Stream (DStream), the basic abstraction in Spark Streaming, 
# is a continuous sequence of RDDs (of the same type) representing a continuous stream of data
# DStreams can either be created from live data (such as, data from TCP sockets, Kafka, Flume, etc.) using a 
# StreamingContext or it can be generated by transforming existing DStreams using operations such as map and window.
# https://blog.jetoile.fr/2014/05/rdd-quest-ce-que-cest.html
ssc.checkpoint("checkpoint")
# Sets the context to periodically checkpoint the DStream operations for master fault-tolerance.
# Parameters:	directory – HDFS-compatible directory where the checkpoint data will be reliably stored
ssc.remember(1)
# Set each DStreams in this context to remember RDDs it generated in the last given duration. 

In [4]:
kvs = KafkaUtils.createStream(ssc, "127.0.0.1:2181", "spark-streaming-consumer", {'volatility': 1})
# createStream(ssc, zkQuorum, groupId, topics)
# Parameters:	
#				ssc – StreamingContext object
#				zkQuorum – Zookeeper quorum (hostname:port,hostname:port,..).
#				groupId – The group id for this consumer.
#				topics – Dict of (topic_name -> numPartitions) to consume. Each partition is consumed in its own thread.
# Topic partitions in Kafka does not correlate to partitions of RDDs generated in Spark Streaming. 

In [5]:
def readInput(line):
    # Convert the input string into a pair of numbers
    print('now')
    vec= [float(x) for x in line.split()] 


    # The pair represents the inputs for two model (forgetting factor 1 and 0.99)
    return [('mod1',('mod1',np.array(vec))),('mod2',('mod2',np.array(vec)))]

In [6]:
def updateFunction(new_values, state): 
    ## RLS update function, formulas slide 51
    ## new_values = (key, [i, y, proxies])
    ## state = what is given by sc.parallelize in next cell on the first call
    ##       = what this function returned on the other calls
    if (len(new_values)>0 ):
        
        key=new_values[0][0]
        yx=new_values[0][1]
        i=yx[0]
        y=yx[1]
        x=yx[2:]
        n=len(x)
        
        beta=state[1]
        beta.shape=(n,1) # Transorms beta in a matrix of n rows and 1 column
        V=state[2]
        mu=state[3]
        sse=state[4]  ## sum of squared errors
        N=state[5]    ## number of treated samples
        x.shape=(1,n)
        err=y-x.dot(beta)        
        sse=sse+pow(err,2.0)
        V=1.0/mu*(V-V.dot(x.T).dot(x).dot(V)/(1.0+float(x.dot(V).dot(x.T)))) # dot = matrix multiplication
                                                                             # .T = Transpose
        gamma=V.dot(x.T)
        beta=beta+gamma*err
        proxyPrediction=x.dot(beta)
        
        errNaivePrec=y-x[0][-1]
        errNaiveMean=y-np.mean(x)
        sseNaivePrec=state[7]
        sseNaiveMean=state[8]
        sseNaivePrec=sseNaivePrec+pow(errNaivePrec,2.0)
        sseNaiveMean=sseNaiveMean+pow(errNaiveMean,2.0)
        
        MSE_RLS=sse/(N+1.0)
        MSE_NaivePrec=sseNaivePrec/(N+1.0)
        MSE_NaiveMean=sseNaiveMean/(N+1.0)
        
        if (key=='mod1'):
            return (key,beta,V,mu,MSE_RLS,N+1, proxyPrediction, sseNaivePrec, sseNaiveMean, MSE_RLS/MSE_NaivePrec, MSE_RLS/MSE_NaiveMean)  ## update formula mod1
        else:
            return (key,beta,V,mu,MSE_RLS,N+1, proxyPrediction, sseNaivePrec, sseNaiveMean, MSE_RLS/MSE_NaivePrec, MSE_RLS/MSE_NaiveMean)  ## update formula mod1
        
    else:
        return state

In [7]:
import re, ast
n=3 # number of features counting t_0
beta1=np.zeros(n)  ## initial parameter vector slide 49
beta2=np.zeros(n)
V1=np.diag(np.zeros(n)+10) ## initial covariance matrix slide 49
V2=np.diag(np.zeros(n)+1)
mu1=1.0 # forgetting factor slide 50
mu2=0.99
data = kvs.map(lambda x: np.array(ast.literal_eval(x[1])))
# map: Returns a new DStream by applying a function to each element of DStream.
# literal_eval: This can be used for safely evaluating strings containing Python values (strings, numbers, tuples, lists, dicts, booleans, and None)
# from untrusted sources without the need to parse the values oneself.
data=data.flatMap(lambda x: [('mod1',('mod1',1.0*np.array(x))),
                            ('mod2',('mod2',1.0*np.array(x)))])
data.pprint() # prints received array
# Return a new DStream by applying a function to all elements of this DStream, and then flattening the results
#initialStateRDD = sc.parallelize([('k',([1,2,3]))])
initialStateRDD = sc.parallelize([(u'mod1', ('mod1',beta1,V1,mu1,0,0,0,0,0)),
                                  (u'mod2', ('mod2',beta2,V2,mu2,0,0,0,0,0))])
# The elements of the collection [] are copied to form a distributed dataset that can be operated on in parallel.
# ('mod1',beta1,V1,mu1,0,0,0)) == what updateFunction returns
data2=data.updateStateByKey(updateFunction,initialRDD=initialStateRDD)
# Return a new "state" DStream where the state for each key is updated by applying the given function on the previous
# state of the key and the new values for the key. This can be used to maintain arbitrary state data for each key.
#data.pprint()
data2.map(lambda x: [x[1][i] for i in [0,1,6,4,9,10]]).pprint() 
# prints key, beta, proxyPrediction, MSE_RLS, NMSE_prec, NMSE_mean

In [8]:
ssc.start()


-------------------------------------------
Time: 2018-03-27 21:32:05
-------------------------------------------

-------------------------------------------
Time: 2018-03-27 21:32:10
-------------------------------------------

-------------------------------------------
Time: 2018-03-27 21:32:15
-------------------------------------------

-------------------------------------------
Time: 2018-03-27 21:32:20
-------------------------------------------

-------------------------------------------
Time: 2018-03-27 21:32:25
-------------------------------------------

-------------------------------------------
Time: 2018-03-27 21:32:30
-------------------------------------------

-------------------------------------------
Time: 2018-03-27 21:32:35
-------------------------------------------

-------------------------------------------
Time: 2018-03-27 21:32:40
-------------------------------------------

-------------------------------------------
Time: 2018-03-27 21:32:45
----------

-------------------------------------------
Time: 2018-03-27 21:36:00
-------------------------------------------
['mod2', array([[ 0.35771161],
       [ 0.03696867],
       [ 0.5269613 ]]), array([[ 1.26150289]]), array([[ 0.51867877]]), array([[ 8.71135079]]), array([[ 1.86935539]])]
['mod1', array([[ 0.41643007],
       [ 0.00959142],
       [ 0.56419877]]), array([[ 1.33291172]]), array([[ 0.51935225]]), array([[ 8.722662]]), array([[ 1.87178265]])]

-------------------------------------------
Time: 2018-03-27 21:36:05
-------------------------------------------
('mod1', ('mod1', array([ 12.        ,   0.61594567,   1.        ,   1.33453846,   0.96464727])))
('mod2', ('mod2', array([ 12.        ,   0.61594567,   1.        ,   1.33453846,   0.96464727])))

-------------------------------------------
Time: 2018-03-27 21:36:05
-------------------------------------------
['mod2', array([[ 0.28632907],
       [ 0.01026233],
       [ 0.55238034]]), array([[ 0.83287672]]), array([[ 0.1520

-------------------------------------------
Time: 2018-03-27 21:36:45
-------------------------------------------
['mod2', array([[-0.01589177],
       [-0.01117881],
       [ 0.77723895]]), array([[ 0.6886672]]), array([[ 0.0408203]]), array([[ 0.25186316]]), array([[ 0.13170869]])]
['mod1', array([[-0.33317093],
       [-0.0563901 ],
       [ 1.04336859]]), array([[ 0.54924538]]), array([[ 0.02375993]]), array([[ 0.14659988]]), array([[ 0.07666257]])]

-------------------------------------------
Time: 2018-03-27 21:36:50
-------------------------------------------
('mod1', ('mod1', array([ 30.        ,  -1.12434726,   1.        ,  -0.87694278,  -1.00138421])))
('mod2', ('mod2', array([ 30.        ,  -1.12434726,   1.        ,  -0.87694278,  -1.00138421])))

-------------------------------------------
Time: 2018-03-27 21:36:50
-------------------------------------------
['mod2', array([[-0.12829643],
       [ 0.00085799],
       [ 0.84137765]]), array([[-0.97159113]]), array([[ 0.0173

-------------------------------------------
Time: 2018-03-27 21:37:30
-------------------------------------------
['mod2', array([[-0.13646955],
       [-0.0050032 ],
       [ 0.85324611]]), array([[-0.37106781]]), array([[ 0.00251577]]), array([[ 0.0230487]]), array([[ 0.00711909]])]
['mod1', array([[-0.17592265],
       [-0.07540798],
       [ 0.96160936]]), array([[-0.42317605]]), array([[ 0.00124653]]), array([[ 0.01142032]]), array([[ 0.00352742]])]

-------------------------------------------
Time: 2018-03-27 21:37:35
-------------------------------------------
('mod1', ('mod1', array([ 47.        ,   1.63724795,   1.        ,   0.72002554,   1.079141  ])))
('mod2', ('mod2', array([ 47.        ,   1.63724795,   1.        ,   0.72002554,   1.079141  ])))
('mod1', ('mod1', array([ 48.        ,   2.13504976,   1.        ,   1.09685822,   1.64735281])))
('mod2', ('mod2', array([ 48.        ,   2.13504976,   1.        ,   1.09685822,   1.64735281])))

---------------------------------

-------------------------------------------
Time: 2018-03-27 21:38:15
-------------------------------------------
['mod2', array([[-0.11270012],
       [-0.04771473],
       [ 0.93355401]]), array([[-1.35771977]]), array([[ 0.00025183]]), array([[ 0.0021983]]), array([[ 0.00061499]])]
['mod1', array([[-0.13448143],
       [-0.11417586],
       [ 1.01691162]]), array([[-1.40326355]]), array([[  3.66319174e-05]]), array([[ 0.00031978]]), array([[  8.94605029e-05]])]

-------------------------------------------
Time: 2018-03-27 21:38:20
-------------------------------------------
('mod1', ('mod1', array([ 65.        ,  -1.2381356 ,   1.        ,  -1.24919039,  -1.24350701])))
('mod2', ('mod2', array([ 65.        ,  -1.2381356 ,   1.        ,  -1.24919039,  -1.24350701])))

-------------------------------------------
Time: 2018-03-27 21:38:20
-------------------------------------------
['mod2', array([[-0.11458574],
       [-0.04716275],
       [ 0.93447168]]), array([[-1.21769257]]), arra

...

-------------------------------------------
Time: 2018-03-27 21:44:00
-------------------------------------------
['mod2', array([[-0.08006792],
       [-0.17605468],
       [ 1.08794532]]), array([[-0.79486966]]), array([[  2.19120553e-06]]), array([[  3.72444417e-05]]), array([[  5.80899494e-06]])]
['mod1', array([[-0.08936759],
       [-0.20508032],
       [ 1.11663159]]), array([[-0.80388947]]), array([[  7.20017493e-06]]), array([[ 0.00012238]]), array([[  1.90880222e-05]])]

-------------------------------------------
Time: 2018-03-27 21:44:05
-------------------------------------------

-------------------------------------------
Time: 2018-03-27 21:44:05
-------------------------------------------
['mod2', array([[-0.08006792],
       [-0.17605468],
       [ 1.08794532]]), array([[-0.79486966]]), array([[  2.19120553e-06]]), array([[  3.72444417e-05]]), array([[  5.80899494e-06]])]
['mod1', array([[-0.08936759],
       [-0.20508032],
       [ 1.11663159]]), array([[-0.80388947]

-------------------------------------------
Time: 2018-03-27 21:44:45
-------------------------------------------
['mod2', array([[-0.07641052],
       [-0.17965671],
       [ 1.09080554]]), array([[-0.22153155]]), array([[  3.00019092e-05]]), array([[ 0.00053963]]), array([[  8.19480844e-05]])]
['mod1', array([[-0.08605345],
       [-0.20742648],
       [ 1.11827556]]), array([[-0.23466928]]), array([[  1.45361831e-05]]), array([[ 0.00026146]]), array([[  3.97045517e-05]])]

-------------------------------------------
Time: 2018-03-27 21:44:50
-------------------------------------------
('mod1', ('mod1', array([ 221.        ,   -0.76117653,    1.        ,   -0.63367535,
         -0.76118814])))
('mod2', ('mod2', array([ 221.        ,   -0.76117653,    1.        ,   -0.63367535,
         -0.76118814])))
('mod1', ('mod1', array([ 222.        ,   -0.7606802 ,    1.        ,   -0.75845054,
         -0.75843862])))
('mod2', ('mod2', array([ 222.        ,   -0.7606802 ,    1.        ,   -0.

-------------------------------------------
Time: 2018-03-27 21:45:30
-------------------------------------------
('mod1', ('mod1', array([ 235.        ,   -0.64781671,    1.        ,   -0.66456658,
         -0.6472893 ])))
('mod2', ('mod2', array([ 235.        ,   -0.64781671,    1.        ,   -0.66456658,
         -0.6472893 ])))
('mod1', ('mod1', array([ 236.        ,   -0.64556204,    1.        ,   -0.64482776,
         -0.64535229])))
('mod2', ('mod2', array([ 236.        ,   -0.64556204,    1.        ,   -0.64482776,
         -0.64535229])))
('mod1', ('mod1', array([ 237.        ,   -0.64332577,    1.        ,   -0.64291051,
         -0.64311709])))
('mod2', ('mod2', array([ 237.        ,   -0.64332577,    1.        ,   -0.64291051,
         -0.64311709])))
('mod1', ('mod1', array([ 238.        ,   -0.64110762,    1.        ,   -0.64069447,
         -0.6409    ])))
('mod2', ('mod2', array([ 238.        ,   -0.64110762,    1.        ,   -0.64069447,
         -0.6409    ])))
('mod1

-------------------------------------------
Time: 2018-03-27 21:46:10
-------------------------------------------

-------------------------------------------
Time: 2018-03-27 21:46:10
-------------------------------------------
['mod2', array([[-0.07034991],
       [-0.18231317],
       [ 1.09248234]]), array([[ 0.01964712]]), array([[  4.14660970e-05]]), array([[ 0.00084814]]), array([[ 0.00011703]])]
['mod1', array([[-0.08056016],
       [-0.20907366],
       [ 1.11856913]]), array([[ 0.01285618]]), array([[  5.17194803e-05]]), array([[ 0.00105786]]), array([[ 0.00014596]])]

-------------------------------------------
Time: 2018-03-27 21:46:15
-------------------------------------------
('mod1', ('mod1', array([ 252.        ,    0.53590562,    1.        ,    0.62860593,
          0.67500909])))
('mod2', ('mod2', array([ 252.        ,    0.53590562,    1.        ,    0.62860593,
          0.67500909])))
('mod1', ('mod1', array([ 253.        ,    1.33242514,    1.        ,    0.67852

-------------------------------------------
Time: 2018-03-27 21:46:50
-------------------------------------------
('mod1', ('mod1', array([ 269.        ,    0.89456017,    1.        ,    1.54861651,
          0.89664868])))
('mod2', ('mod2', array([ 269.        ,    0.89456017,    1.        ,    1.54861651,
          0.89664868])))
('mod1', ('mod1', array([ 270.        ,    0.50593648,    1.        ,    0.90006238,
          0.89797954])))
('mod2', ('mod2', array([ 270.        ,    0.50593648,    1.        ,    0.90006238,
          0.89797954])))

-------------------------------------------
Time: 2018-03-27 21:46:50
-------------------------------------------
['mod2', array([[-0.05469477],
       [-0.23963203],
       [ 1.17327135]]), array([[ 0.62621932]]), array([[ 0.001153]]), array([[ 0.01849833]]), array([[ 0.00296057]])]
['mod1', array([[-0.06809261],
       [-0.24077863],
       [ 1.17091183]]), array([[ 0.60893017]]), array([[ 0.00121811]]), array([[ 0.01954291]]), array([[ 0.

In [9]:
ssc.stop(stopSparkContext=False,stopGraceFully=True)

-------------------------------------------
Time: 2018-03-19 17:54:25
-------------------------------------------

-------------------------------------------
Time: 2018-03-19 17:54:25
-------------------------------------------
['mod2', array([[  1.14151631e-08],
       [  2.17159392e-15],
       [  2.18195076e-15]]), array([[  1.14151631e-08]]), array([[  1.30719998e-17]]), array([[ 1.02368855]]), array([[  1.17648000e-16]])]
['mod1', array([[  1.26100554e-08],
       [  1.86921790e-14],
       [  1.90980643e-14]]), array([[  1.26100554e-08]]), array([[  1.59337649e-17]]), array([[ 1.2477978]]), array([[  1.43403885e-16]])]

-------------------------------------------
Time: 2018-03-19 17:54:30
-------------------------------------------

-------------------------------------------
Time: 2018-03-19 17:54:30
-------------------------------------------
['mod2', array([[  1.14151631e-08],
       [  2.17159392e-15],
       [  2.18195076e-15]]), array([[  1.14151631e-08]]), array([[  1.307

In [ ]:
a=[1,2,3,4]


In [ ]:
[a[i] for i in [1,0,2,3]]
